# The Iterable Response Requests

## Table of Contents

## Overview of the Iter... CIM_Operations

### Why
As of PyWBEM 10, a new set of methods was added to the WBEMConnection methods that provide an overlay over the
Pywbem Pull operations to provide:
    
1. An interface that is consistent whether the user is executing the Pull Operations or their equivalent not pull operations

2. An interface that use the Python iterable interface to get instances or paths from the response in place of lists or
tuples as required for the Pull and non_pull operations.

3. An interface that allows the user to utilize pull or non-pull operations with just an attribute change in WBEMConnection

4. An interface that automatically attempts to use Pull Operations and if a particular WBEM Server does not support them falls back
to the non-pull equivalent operations

### Request Comparison Table

The CIM/XML pull operations parallel to the previous CIM/XML operations as follows

*Original Operation*   | *Equivalent Pull Operations*                     | Iter Operations
 :-                    |:-                                                | :-
EnumerateInstances     | OpenEnumerateInstances / PullInstancesWithPath   | IterEnumerateInstances
EnumerateInstanceNames | OpenEnumerateInstancePaths / PullInstancesPaths  | InterEnumerateInstancePaths
Associators            | OpenAssociatorInstances / PullInstancesWithPath  | IterAssociatorInstances
AssociatorNames        | OpenAssociatorPaths / PullInstancesPaths         | IterAssociatorPaths
References             | OpenReferenceInstances / PullInstancesWithPath   | IterReferenceInstances
ReferenceNames         | OpenReferencesPaths / PullInstancesPaths         | IterReferencePaths
ExecQuery              | OpenQueryInstances / PullInstances               | IterQueryInstances

The Iter... operation requests use the same request parameters as the Open... operations with the following exceptions:

1. If MaxObjectCount is not supplied it is set from a default to a non-zero integer since this is the MaxObjectCount used both by the Open and the Pull operations.

The general pattern for use of these operations is:

::
    try:
        iterator = Iter(...)
        for object in iterator:
            . .. process the object returned
    except Error as er:
    
These operations use the Python generator paradigm so that the for loop processes response objects as then are received
via Open and Pull operations

### Internal Paradigm

These operations use the existing Open/Pull or original requests and lay a layer over them to determine if the
pull operations can be used and to manage the iteration

   try:
       result = Open...
          for object in result.object
              yield object
          while not result.eos
              result = Pull...
                  for object in result.object:
                      yield object
    except CIMError as ce
        if ce.status_code |= "CIMERR_NOT_SUPPORTED:
        raise ce
    else:
        <originalOperation>(...)

### Forcing Pull vs NonPull operations

A parameter has been added to the [`pywbem.WBEMConnection`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection) constructor to optionally force the use of either the pull
operations or the original operations.

* If `use_pull_operations` is set to True only the pull operation will be executed and if this fails for any
reason including  `CIM_ERR_NOT_SUPPORTED` the exception will be returned.

* If `use_pull_operations` is set to False only the original operation will be executed and if this fails for any
reason, the exception will be returned.

* The default is `None`. In this case, first the pull sequence is attempted. If the first request returns `CIM_ERR_NOT_SUPPORTED`, the corresponding non-pull request will be attempted.

This parameter `use_pull_operations` defaults to `None`.

Thus, the Iter operations can be used to execute exclusively the original operations by simply setting `use_pull_operations = False`.

::

    conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True,
                             use_pull_operations=False)



### Differences between Iter operations and the lower level operations

### Use of QueryFilters

Since the original operations did not incorporate the query filters, if a query filter is included in the request
and the request is passed to the original operation, the request will be refused and an exception generated.

### Response Paths

Since the response paths requirements on returned instances differe between pull and non-pull operations the data the client code acts to bring the path into line with the requirements of the pull operations responses if the original
operation was executed.

### Use of MaxObjectCount

The MaxObjectCount is somewhat more limited than if the pull operations are used directly in that:

1. It is the same value for open and pull requests
2. The mechanism to delay responses (setting MaxObjectCount=0 and executing a Pull request) cannot be used so the interoperation timeout must be sufficient for the client to complete its requests.

### Closing a pull operation before it is complete

An operation may be closed before the processing from the server is complete by executing the generator `close()`
function:

::

    iterator = conn.IterEnumerateInstances(classname, MaxObjectCount=max_obj_cnt)
    for inst in iterator
    if <fails some test>
        insts_iterator.close()
    else:
        <process instance>

Note that if the operation executed was the EnumerateInstances rather than the pull operations, the close() will do
nothing since the response instances are received as a single block.


## IterEnumerateInstances()

The following code executes the iterator cim operation [`IterEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterEnumerateInstances)
to enumerate instances of a class. 

The operation returns an iterator.  The for loop processes that iterator.  Since the iterator is actually a Python generator it waits in the WBEMConnection.IterEnumerateInstances() method until the next instance is available from
the server.
                                                                                    

In [ ]:
from __future__ import print_function
import pywbem

username = 'user'
password = 'password'
classname = 'CIM_ComputerSystem'
namespace = 'root/cimv2'
server = 'http://localhost'
max_obj_cnt = 100

from __future__ import print_function
import pywbem

username = 'user'
password = 'password'
classname = 'CIM_ComputerSystem'
namespace = 'root/cimv2'
server = 'http://localhost'
max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    iterator = conn.IterEnumerateInstances(classname, MaxObjectCount=max_obj_cnt)
    for instance in iterator:
        print('path=%s' % instance.path)
        print(instance.tomof())                                 
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

In this example (and the examples below), the connection has a default namespace set. This allows us to omit the namespace from the subsequent [`IterEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.IterEnumerateInstances) method.

This example also shows exception handling with PyWBEM: PyWBEM wraps any exceptions that are considered runtime errors, and raises them as subclasses of [`pywbem.Error`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.Error). Any other exceptions are considered programming errors. Therefore, the code above only needs to catch [`pywbem.Error`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.Error).

Note that the creation of the [`pywbem.WBEMConnection`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection) object in the code above does not need to be protected by exception handling; its initialization code does not raise any PyWBEM runtime errors.

## IterEnumerateInstancePaths()

In [ ]:
Documentation

In [ ]:
#  Globals are imported from the first example

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    iterator = conn.IterEnumerateInstancePaths(classname, MaxObjectCount=max_obj_cnt)
    for path in iterator:
        print('path=%s' % path)                               
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

In [ ]:
## IterAssociatorInstances

In [ ]:
TODO doc

In [ ]:
TODO Code

In [ ]:
## IterAssociatorPaths

In [ ]:
## IterReferenceInstances

In [ ]:
##IterReferencePaths

In [ ]:
##IterQueryInstances

In [ ]:
## Iter.. requests and list comprehension

In addition to the for loop processor for getting object from the iterator returned by the Iter... call, a list
comprehension can be use to perform the entire processing sequence in a single statement.

In [ ]:
try:
    paths = [path for path in conn.IterEnumerateInstancePaths(classname, MaxObjectCount=max_obj_cnt)]
    print(*paths, sep='\n')

except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

In [ ]:
## Iter... Forcing use of Pull Operations

In [ ]:
TODO example of forcing particular operation type